<h1> Notebook for Infrastructure as Code (IAC)<h1>

<h3>Spin-up Redshift Cluster</h3>

In [1]:
import configparser
import boto3
import pandas as pd

In [2]:
#Remove Key and Secret from dwh.cfg file before submitting
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

<h3>Define parameters for redshift</h3>

In [3]:
redshift = boto3.client('redshift',
                        region_name="us-west-2",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                       )

<h3>Create the cluster</h3>

In [4]:
try:
    response = redshift.create_cluster(        
        #Add parameters for hardware
        ClusterType='single-node',
        NodeType='dc2.large',
        #NumberOfNodes=1,
        

        #Add parameters for identifiers & credentials
        DBName='my_udacity_db',
        ClusterIdentifier='my-udacity-db',
        MasterUsername='dbuser',
        MasterUserPassword='DB_Password2',
        
        #Add parameter for role (to allow s3 access)
        IamRoles=['arn:aws:iam::737033187011:role/myRedshiftRole']
         
    )
except Exception as e:
    print(e)

<h3> Describe Cluster to see it's Status</h3>

In [6]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier='my-udacity-db')['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,my-udacity-db
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dbuser
4,DBName,my_udacity_db
5,Endpoint,"{'Address': 'my-udacity-db.crd2lwlmrmzb.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-7e7bd506
7,NumberOfNodes,1


<h3>Delete Cluster</h3>

In [7]:
#### CAREFUL!!
#Run to delete cluster
redshift.delete_cluster( ClusterIdentifier='my-udacity-db',  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'my-udacity-db',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dbuser',
  'DBName': 'my_udacity_db',
  'Endpoint': {'Address': 'my-udacity-db.crd2lwlmrmzb.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2019, 5, 21, 17, 20, 0, 510000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-7e7bd506',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'sun:08:00-sun:08:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 1,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'IamRoles': [{'IamRoleArn': 'arn:aws:iam::73703318

<h3>Check status of cluster deletion</h3>

In [44]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier='my-udacity-db')['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,my-udacity-db
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dbuser
4,DBName,my_udacity_db
5,Endpoint,"{'Address': 'my-udacity-db.crd2lwlmrmzb.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-7e7bd506
7,NumberOfNodes,1
